Extrae el % de bateria del comentario y lo pega en la columna de bateria

In [1]:
import os
import pandas as pd
from datetime import datetime
from utils.feature_engineering.iphone import *

In [2]:
elemento_a_buscar = "iphone"
fecha = datetime.now().strftime("%Y%m%d")
carpeta_origen = "../data/4_download_description"
carpeta_destino = "../data/5_data_from_comments"

In [3]:
def get_data_from_comments(n):
    n=str(n)
    filepath = os.path.join(carpeta_origen, f"{elemento_a_buscar}_{fecha}_{n}.csv")
    # Si el archivo no existe para
    if not os.path.exists(filepath):
        return None
    df_original = pd.read_csv(filepath)
    df_original.drop('Unnamed: 0', axis=1, inplace=True)
    #df_original.head(2)
    df = df_original.copy()
    missing_values = df_original.isnull().sum()
    print('missing_values', missing_values)
    df['bateria'] = df_original['comentario'].apply(encontrar_bateria)
    df['memoria'] = df_original['comentario'].apply(encontrar_memoria)
    df['tienda'] = df_original['comentario'].apply(es_tienda)
    df['garantia'] = df_original['comentario'].apply(tiene_garantia)
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    df.to_csv(os.path.join(carpeta_destino, f"{fecha}_{elemento_a_buscar}_{n}.csv"))

    missing_values = df_original.isnull().sum()
    print('df_original', missing_values)
    missing_values = df.isnull().sum()
    print("df missing values: ", missing_values)

In [4]:
for n in range(10):
    get_data_from_comments(n)

missing_values id                 0
time_scrap         0
gen                0
mod                0
memoria           30
bateria           90
precio             0
estado             0
nombre             0
tiene_color        0
tiene_emojis       0
tiene_revisado     0
municipio          0
distancia          0
reservado          0
url_articulo       0
fecha              0
comentario         8
dtype: int64
df_original id                 0
time_scrap         0
gen                0
mod                0
memoria           30
bateria           90
precio             0
estado             0
nombre             0
tiene_color        0
tiene_emojis       0
tiene_revisado     0
municipio          0
distancia          0
reservado          0
url_articulo       0
fecha              0
comentario         8
dtype: int64
df missing values:  id                 0
time_scrap         0
gen                0
mod                0
memoria           14
bateria           39
precio             0
estado             0
nom